# Sitemap Assay

The start of a simple notebook that could hosted for peeopl to test out their sitemaps (and robots.txt) files with.

References:
* [AdvTools](https://advertools.readthedocs.io/en/master/advertools.sitemaps.html)
* [Sitemap viz](https://www.ayima.com/us/insights/analytics-and-cro/how-to-visualize-an-xml-sitemap-using-python.html)


<a href="https://githubtocolab.com/gee-community/geemap/blob/master/examples/notebooks/35_geemap_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open in Colab"/></a>



## Notes

has json-ld :  https://researchdata.edu.au/imos-soop-underway-sep-2017/970828
Validation:  https://validator.schema.org/#url=https%3A%2F%2Fresearchdata.edu.au%2Fimos-soop-underway-sep-2017%2F970828  


doesn't:  https://researchdata.edu.au/heupel-michelle/1709766

Validation   https://validator.schema.org/#url=https%3A%2F%2Fresearchdata.edu.au%2Fheupel-michelle%2F1709766


Context as map:  https://www.w3.org/TR/json-ld/#context-definitions





In [1]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

import advertools as adv
import json
import requests
from pyld import jsonld
from bs4 import BeautifulSoup
import urllib.request
import logging
import traceback
import kglab
import pandas as pd


In [2]:
%%time 

# smurl = 'https://opencoredata.org/sitemap_0.xml'
# smurl = 'https://www.bco-dmo.org/sitemap.xml'
# smurl = 'https://opentopography.org/sitemap.xml'
# smurl = 'https://www2.earthref.org/MagIC/contributions.sitemap.xml'
# smurl = 'https://oceanscape.org/organisation-sitemap.xml'
# smurl = 'https://catalogue.cioosatlantic.ca/sitemap/sitemap-1.xml'
# smurl = 'https://obis.org/sitemap/sitemap_datasets.xml'
# smurl = 'https://infohub.eurocean.net/sitemap/vessels'
# smurl = 'https://raw.githubusercontent.com/iodepo/odis-arch/schema-dev-jm/code/notebooks/Exploration/data-pacificdatahub/sitemap.xml'
# smurl = 'https://catalogue.cioospacific.ca/sitemap/sitemap-1.xml'
# smurl = 'https://geonode.nodc.id/sitemap.xml'
# smurl = "https://lipdverse.org/sitemap.xml"
# smurl = "https://pacificdata.org/organization/sitemap.xml"
# smurl = "https://pacificdata.org/data/sitemap/cookislands-data/sitemap.xml"
# smurl = 'https://geoconnex.us/sitemap/ref/states/states__0.xml'
# smurl = "https://oceanscape.org/organisation-sitemap.xml"
# smurl = "https://catalogue.cioos.ca/sitemap/sitemap.xml"
# smurl = "https://geoconnex.us/sitemap.xml"
# smurl = "https://ioc-africa.org/sitemap.xml"
# smurl = "https://researchdata.edu.au/home/sitemap/?ds=64"
# smurl = "https://uatweb.spc.links.com.au/sitemap.xml"
# smurl = "https://uatweb.spc.links.com.au/sitemap.xml"
# smurl = "https://pacificdata.org/organization/sitemap.xml"
# smurl = "https://raw.githubusercontent.com/BeBOP-OBON/odis-interface/main/sitemap.xml"
# smurl = "https://researchdata.edu.au/home/sitemap/?ds=64"
# smurl = "https://catalogue.cioos.ca/sitemap/sitemap.xml"
# smurl = "https://api.linked-systems.uk/sitemap_pap_api.xml"
# smurl = "https://pacific-data.sprep.org/sitemap.xml"
# smurl = "https://metadata.naturalsciences.be/geonetwork/srv/api/sitemap"
# smurl = "https://emodnet.ec.europa.eu/geonetwork/srv/eng/portal.sitemap"
smurl = "https://raw.githubusercontent.com/LTREB-reservoirs/vera4cast/gh-pages/sitemap.xml"

iow_sitemap = adv.sitemap_to_df(smurl) # load sitemap to dataframe via advertools
# iow_sitemap.info()
# iow_sitemap.head()

2023-10-12 11:10:27,003 | INFO | sitemaps.py:536 | sitemap_to_df | Getting https://raw.githubusercontent.com/LTREB-reservoirs/vera4cast/gh-pages/sitemap.xml


CPU times: user 28.2 ms, sys: 2.13 ms, total: 30.3 ms
Wall time: 257 ms


## Analyzing the URLs

We can quickly grab the unique URLs from the sitemap column and see how many unique sitemap.xml files we are working with

We can also dive into the URL structure for the resources a bit.

In [3]:
usm = iow_sitemap.sitemap.unique()
uloc = iow_sitemap["loc"].unique()
print("{} unique sitemap XML file(s) pointing to {} unique resource(s).".format(len(usm), len(uloc)))


1 unique sitemap XML file(s) pointing to 29 unique resource(s).


In [4]:

from urllib.parse import urlparse

invalid_urls = []

for url in uloc:
    try:
        result = urlparse(url)
        if all([result.scheme, result.netloc]):
            pass
        else:
            invalid_urls.append(url)
    except ValueError:
        print(f"{url} is an exception URL")
        invalid_urls.append(url)


df = pd.DataFrame({'Invalid URLs': invalid_urls})


In [5]:
df.head()

,Invalid URLs


In [6]:
from urllib.parse import urlparse

valid_urls = []


for url in uloc:
    try:
        result = urlparse(url)
        if all([result.scheme, result.netloc]):
            valid_urls.append(url)
        else:
            print(f"Invalid URL: {url}")
    except ValueError:
        print(f"Invalid URL: {url}")


In [7]:

# Break down all the URL into their path parts
urldf = adv.url_to_df(list(iow_sitemap['loc']))
# urldf = adv.url_to_df(list(uloc))

urldf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 29 entries, 0 to 28
Data columns (total 10 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   url       29 non-null     object 
 1   scheme    29 non-null     object 
 2   netloc    29 non-null     object 
 3   path      29 non-null     object 
 4   query     0 non-null      float64
 5   fragment  25 non-null     object 
 6   hostname  4 non-null      object 
 7   dir_1     29 non-null     object 
 8   dir_2     4 non-null      object 
 9   last_dir  29 non-null     object 
dtypes: float64(1), object(9)
memory usage: 2.4+ KB


In [8]:
urldf.head()


,url,scheme,netloc,path,query,fragment,hostname,dir_1,dir_2,last_dir
0,https://LTREB-reservoirs.github.io/vera4cast/c...,https,LTREB-reservoirs.github.io,/vera4cast/catalog.html,NaN,NaN,ltreb-reservoirs.github.io,vera4cast,catalog.html,catalog.html
1,https://LTREB-reservoirs.github.io/vera4cast/i...,https,LTREB-reservoirs.github.io,/vera4cast/instructions.html,NaN,NaN,ltreb-reservoirs.github.io,vera4cast,instructions.html,instructions.html
2,https://LTREB-reservoirs.github.io/vera4cast/d...,https,LTREB-reservoirs.github.io,/vera4cast/daily.html,NaN,NaN,ltreb-reservoirs.github.io,vera4cast,daily.html,daily.html
3,https://LTREB-reservoirs.github.io/vera4cast/i...,https,LTREB-reservoirs.github.io,/vera4cast/index.html,NaN,NaN,ltreb-reservoirs.github.io,vera4cast,index.html,index.html
4,https://radiantearth.github.io/stac-browser/#/...,https,radiantearth.github.io,/stac-browser/,NaN,/external/raw.githubusercontent.com/LTREB-rese...,NaN,stac-browser,NaN,stac-browser


## Sample and test sitemap entries

In [9]:
# sample the previously generated url data frame
sample_size = 5
# sample_df = urldf.groupby("dir_1").sample(n=sample_size, random_state=1, replace=True)
sample_df = urldf.sample(n=sample_size, random_state=1, replace=True)

In [10]:
sample_df.head(5)

,url,scheme,netloc,path,query,fragment,hostname,dir_1,dir_2,last_dir
5,https://radiantearth.github.io/stac-browser/#/...,https,radiantearth.github.io,/stac-browser/,NaN,/external/raw.githubusercontent.com/LTREB-rese...,NaN,stac-browser,NaN,stac-browser
11,https://radiantearth.github.io/stac-browser/#/...,https,radiantearth.github.io,/stac-browser/,NaN,/external/raw.githubusercontent.com/LTREB-rese...,NaN,stac-browser,NaN,stac-browser
12,https://radiantearth.github.io/stac-browser/#/...,https,radiantearth.github.io,/stac-browser/,NaN,/external/raw.githubusercontent.com/LTREB-rese...,NaN,stac-browser,NaN,stac-browser
8,https://radiantearth.github.io/stac-browser/#/...,https,radiantearth.github.io,/stac-browser/,NaN,/external/raw.githubusercontent.com/LTREB-rese...,NaN,stac-browser,NaN,stac-browser
9,https://radiantearth.github.io/stac-browser/#/...,https,radiantearth.github.io,/stac-browser/,NaN,/external/raw.githubusercontent.com/LTREB-rese...,NaN,stac-browser,NaN,stac-browser


### See if the URLs resolve

In [11]:
import urllib.request
import requests

ul = sample_df["url"]

for item in ul:
    # user_agent = 'Mozilla/5.0 (Windows; U; Windows NT 5.1; en-US; rv:1.9.0.7) Gecko/2009021910 Firefox/3.0.7'
    # headers={'User-Agent':user_agent,}

    headers= {'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) '
                           'AppleWebKit/537.11 (KHTML, like Gecko) '
                           'Chrome/23.0.1271.64 Safari/537.11',
             'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8',
             'Accept-Charset': 'ISO-8859-1,utf-8;q=0.7,*;q=0.3',
             'Accept-Encoding': 'none',
             'Accept-Language': 'en-US,en;q=0.8',
             'Connection': 'keep-alive'}

    try:
        # x = requests.get(item)
        # code = x.status_code
        request=urllib.request.Request(url=item, headers=headers) #The assembled request
        with urllib.request.urlopen(request) as response:
            info = response.info()
            dtype = info.get_content_type()    # -> text/html
         # headers = x.headers()
        # print("URL: {} \ninfo : {}\n --".format(item, info))
        print("URL: {} ".format(item))
    except Exception as e:
        # code = x.status_code
        # dtype = info.get_content_type()

        print("Exception on: {} \nerrors : {}\n --".format(item, str(e)))


URL: https://radiantearth.github.io/stac-browser/#/external/raw.githubusercontent.com/LTREB-reservoirs/vera4cast/main/catalog/forecasts/Biological/Chla_ugL_mean/collection.json 
URL: https://radiantearth.github.io/stac-browser/#/external/raw.githubusercontent.com/LTREB-reservoirs/vera4cast/main/catalog/forecasts/models/model_items/TESTclimatology.json 
URL: https://radiantearth.github.io/stac-browser/#/external/raw.githubusercontent.com/LTREB-reservoirs/vera4cast/main/catalog/forecasts/models/model_items/climatology.json 
URL: https://radiantearth.github.io/stac-browser/#/external/raw.githubusercontent.com/LTREB-reservoirs/vera4cast/main/catalog/forecasts/Physical/collection.json 
URL: https://radiantearth.github.io/stac-browser/#/external/raw.githubusercontent.com/LTREB-reservoirs/vera4cast/main/catalog/forecasts/collection.json 


### See if they have JSON-LD (static check only, no dynamically loaded JSON-LD yet)

In [12]:
ul = sample_df["url"]

headers= {'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) '
                        'AppleWebKit/537.11 (KHTML, like Gecko) '
                        'Chrome/23.0.1271.64 Safari/537.11',
          'Accept': 'application/ld+json,text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8',
          'Accept-Charset': 'ISO-8859-1,utf-8;q=0.7,*;q=0.3',
          'Accept-Encoding': 'none',
          'Accept-Language': 'en-US,en;q=0.8',
          'Connection': 'keep-alive'}

for item in ul:
    print(item)
    request=urllib.request.Request(url=item, headers=headers)
    # p = urllib.request.urlopen(request).read()
    
    html = urllib.request.urlopen(request).read()
    soup = BeautifulSoup(html, "html.parser")
    p = soup.find('script', {'type':'application/ld+json'})
    try:
        # print("JSON byte size: {} ".format(len(p)))
        print(p)
        print("JSON byte size: {} ".format(len(p.contents[0])))
    except Exception as e:
        logging.error(traceback.format_exc())

2023-10-12 11:10:45,267 | ERROR | <ipython-input-12-7deccc5a4788>:25 | <module> | Traceback (most recent call last):
  File "<ipython-input-12-7deccc5a4788>", line 23, in <module>
    print("JSON byte size: {} ".format(len(p.contents[0])))
AttributeError: 'NoneType' object has no attribute 'contents'

2023-10-12 11:10:45,321 | ERROR | <ipython-input-12-7deccc5a4788>:25 | <module> | Traceback (most recent call last):
  File "<ipython-input-12-7deccc5a4788>", line 23, in <module>
    print("JSON byte size: {} ".format(len(p.contents[0])))
AttributeError: 'NoneType' object has no attribute 'contents'

2023-10-12 11:10:45,375 | ERROR | <ipython-input-12-7deccc5a4788>:25 | <module> | Traceback (most recent call last):
  File "<ipython-input-12-7deccc5a4788>", line 23, in <module>
    print("JSON byte size: {} ".format(len(p.contents[0])))
AttributeError: 'NoneType' object has no attribute 'contents'



https://radiantearth.github.io/stac-browser/#/external/raw.githubusercontent.com/LTREB-reservoirs/vera4cast/main/catalog/forecasts/Biological/Chla_ugL_mean/collection.json
None
https://radiantearth.github.io/stac-browser/#/external/raw.githubusercontent.com/LTREB-reservoirs/vera4cast/main/catalog/forecasts/models/model_items/TESTclimatology.json
None
https://radiantearth.github.io/stac-browser/#/external/raw.githubusercontent.com/LTREB-reservoirs/vera4cast/main/catalog/forecasts/models/model_items/climatology.json
None
https://radiantearth.github.io/stac-browser/#/external/raw.githubusercontent.com/LTREB-reservoirs/vera4cast/main/catalog/forecasts/Physical/collection.json


2023-10-12 11:10:45,430 | ERROR | <ipython-input-12-7deccc5a4788>:25 | <module> | Traceback (most recent call last):
  File "<ipython-input-12-7deccc5a4788>", line 23, in <module>
    print("JSON byte size: {} ".format(len(p.contents[0])))
AttributeError: 'NoneType' object has no attribute 'contents'

2023-10-12 11:10:45,484 | ERROR | <ipython-input-12-7deccc5a4788>:25 | <module> | Traceback (most recent call last):
  File "<ipython-input-12-7deccc5a4788>", line 23, in <module>
    print("JSON byte size: {} ".format(len(p.contents[0])))
AttributeError: 'NoneType' object has no attribute 'contents'



None
https://radiantearth.github.io/stac-browser/#/external/raw.githubusercontent.com/LTREB-reservoirs/vera4cast/main/catalog/forecasts/collection.json
None


### Check JSON-LD structure (static check only, no dynamically loaded JSON-LD yet)

In [13]:
ul = sample_df["url"]

myframe =  {
    "@context":{"@vocab": "http://schema.org/"},
    "@type": "Dataset",
}

context =  { "@vocab": "http://schema.org/" }

headers= {'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) '
                        'AppleWebKit/537.11 (KHTML, like Gecko) '
                        'Chrome/23.0.1271.64 Safari/537.11',
          'Accept': 'application/ld+json,text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8',
          'Accept-Charset': 'ISO-8859-1,utf-8;q=0.7,*;q=0.3',
          'Accept-Encoding': 'none',
          'Accept-Language': 'en-US,en;q=0.8',
          'Connection': 'keep-alive'}

for item in ul:
    request=urllib.request.Request(url=item, headers=headers)
    # p = urllib.request.urlopen(request).read()
    html = urllib.request.urlopen(request).read()
    soup = BeautifulSoup(html, "html.parser")
    p = soup.find('script', {'type':'application/ld+json'})
    try:
        jld = json.loads(p.contents[0])
        # jld = json.loads(p)

        # print(str(jld))
        # compacted = jsonld.compact(str(jld), context)
        # print(len(json.dumps(compacted, indent=2)))
    except Exception as e:
        print("Exception")
        logging.error(traceback.format_exc())

2023-10-12 11:10:50,151 | ERROR | <ipython-input-13-9d11dcc3fcb5>:34 | <module> | Traceback (most recent call last):
  File "<ipython-input-13-9d11dcc3fcb5>", line 26, in <module>
    jld = json.loads(p.contents[0])
AttributeError: 'NoneType' object has no attribute 'contents'

2023-10-12 11:10:50,205 | ERROR | <ipython-input-13-9d11dcc3fcb5>:34 | <module> | Traceback (most recent call last):
  File "<ipython-input-13-9d11dcc3fcb5>", line 26, in <module>
    jld = json.loads(p.contents[0])
AttributeError: 'NoneType' object has no attribute 'contents'

2023-10-12 11:10:50,257 | ERROR | <ipython-input-13-9d11dcc3fcb5>:34 | <module> | Traceback (most recent call last):
  File "<ipython-input-13-9d11dcc3fcb5>", line 26, in <module>
    jld = json.loads(p.contents[0])
AttributeError: 'NoneType' object has no attribute 'contents'

2023-10-12 11:10:50,310 | ERROR | <ipython-input-13-9d11dcc3fcb5>:34 | <module> | Traceback (most recent call last):
  File "<ipython-input-13-9d11dcc3fcb5>", line

Exception
Exception
Exception
Exception


2023-10-12 11:10:50,363 | ERROR | <ipython-input-13-9d11dcc3fcb5>:34 | <module> | Traceback (most recent call last):
  File "<ipython-input-13-9d11dcc3fcb5>", line 26, in <module>
    jld = json.loads(p.contents[0])
AttributeError: 'NoneType' object has no attribute 'contents'



Exception


## Load to Graph"

Looad a sample set of triples into RDF lib and run a sample SPARQL query on them.

### Note
This is the same loop as above but now we will load into a KG graph

In [19]:
ul = sample_df["url"]

# Test loading into a graph
namespaces = {
    "schema":  "http://schema.org/",
    "schemaold":  "http://schema.org/",
    "shacl":   "http://www.w3.org/ns/shacl#" ,
}

kg = kglab.KnowledgeGraph(
    name = "Schema.org shacl eval datagraph",
    base_uri = "https://example.org/id/",
    namespaces = namespaces,
)

for item in ul:
    html = urllib.request.urlopen(item).read()
    soup = BeautifulSoup(html, "html.parser")
    p = soup.find('script', {'type':'application/ld+json'})
    try:
        print("JSON byte size: {} ".format(len(p.contents[0])))
        kg.load_rdf_text(data=p.contents[0], format="json-ld")
        print(p.contents[0])
    except Exception as e:
        logging.error(traceback.format_exc())

HTTPError: HTTP Error 403: Forbidden

In [20]:
sparql = """
PREFIX schema: <http://schema.org/>
SELECT ?s ?name ?description ?type
  WHERE {
    ?s a ?type .    
    ?s schema:name ?name .
    ?s schema:description ?description.
  }
"""

#  schema:Dataset 
# ?s schema:name ?name .
#     ?s schema:description ?description.

df = kg.query_as_df(sparql).to_pandas()

df.head()

AttributeError: 'DataFrame' object has no attribute 'to_pandas'